## Import Libraries

In [1]:
from os import path, makedirs
from itertools import product
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
import scipy.io as sio

from brainpipe.classification import *
from brainpipe.system import study
from brainpipe.feature import power, amplitude, sigfilt
from brainpipe.visual import *
from brainpipe.statistics import *
from scipy.stats import *

## Power Decoding - Partial//Detailed Encoding
### For ALL time points

In [4]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold as SKFold
from sklearn.model_selection import permutation_test_score
from sklearn.metrics import roc_auc_score
from numpy.random import permutation

st = study('Olfacto')
exp = 'Ret'#'Enc'
conds, subjects = ['low','high'],['VACJ','CHAF','SEMC','LEFC','PIRJ','FERJ']
freqs = ['0_theta','1_alpha','2_beta','3_gamma']
path_pow = path.join(st.path, 'feature/TPSim_power_data/Power_all_elecs_E_R_by_cond/')
pow_name = path.join(path_pow, 'Pow_{}_{}_{}_allfreqs.npz')
save_path = path.join(st.path, 'classified/0_Classif_Pow_sklearn_mean_'+exp[0]+'_by_cond_1_2s/')
clf_name = path.join(save_path, '{}_LDA_clf_{}_{}_{}.npz')
nperm = 1000

for su in subjects:
    for i,freq in enumerate(freqs):
        pow_list = []
        #=========================== Load Power files (nfreq, nelec, nwin, ntrial) =================================    
        mat0 = np.load(pow_name.format(su,exp[0],conds[0]), allow_pickle=True)
        pow_list.append(np.mean(mat0['xpow'][i,:,37:47,:],axis=1))
        nelecs = mat0['xpow'][i,:,37:47,:].shape[0]
        mat1 = np.load(pow_name.format(su,exp[0],conds[1]), allow_pickle=True)
        pow_list.append(np.mean(mat1['xpow'][i,:,37:47,:],axis=1))
        print (su,mat0.files, 'xpow shape: ', [pow.shape for pow in pow_list])
        
        #=========================== Create dict for all results =================================    
        name_classif = clf_name.format(su,freq,conds[0],conds[1])
        if not path.exists(name_classif):
            kwargs = {}
            kwargs['names'], kwargs['channels'] = mat0['label'], mat0['channel']
            kwargs['xyz'] = mat0['xyz']

            # =========================== Select Power for 1 elec 1 freq =================================                 
            permut,auc = np.zeros((nelecs,nperm)),np.zeros((nelecs))
            for elec_num in range(nelecs):
                print('--» processing',su, 'elec', elec_num,'/',nelecs, 'freq',freq)
                pow_data_elec = [power[elec_num][:,np.newaxis] for power in pow_list]

                # create a data matrix, concatenate along the trial dimension
                x = np.concatenate(pow_data_elec, axis=0)
                print ('Size of the concatenated data: ', x.shape)
                y = np.hstack([np.array([i]*len(power)) for i, power in enumerate(pow_data_elec)])
                print ('Size of label for classif: ', len(y))

                clf = LDA()
                cv = SKFold(5)
                score, permutation_scores, pvalue = permutation_test_score(
                clf, x, y, scoring="roc_auc", cv=cv, n_permutations=nperm, n_jobs=-1)
                permut[elec_num] += permutation_scores
                auc[elec_num] += score
                print("Classification score %s (pvalue : %s)" % (score, pvalue))

            kwargs['perm'], kwargs['auc'] = permut, auc
            #Save plots
            np.savez(name_classif, **kwargs)
            del x, y, pow_data_elec, permutation_scores, score
        else:
            print(name_classif, 'already computed')
    del pow_list

-> Olfacto loaded
VACJ ['xpow', 'label', 'channel', 'xyz'] xpow shape:  [(48, 12), (48, 12)]
--» processing VACJ elec 0 / 48 freq 0_theta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.55 (pvalue : 0.47352647352647353)
--» processing VACJ elec 1 / 48 freq 0_theta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.4722222222222222 (pvalue : 0.5114885114885115)
--» processing VACJ elec 2 / 48 freq 0_theta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.8 (pvalue : 0.01898101898101898)
--» processing VACJ elec 3 / 48 freq 0_theta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.6444444444444445 (pvalue : 0.3116883116883117)
--» processing VACJ elec 4 / 48 freq 0_theta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.7722222222222223 (pvalue : 0.030969030969030968)
--

Classification score 0.4222222222222222 (pvalue : 0.5614385614385614)
--» processing VACJ elec 46 / 48 freq 0_theta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.6555555555555556 (pvalue : 0.24675324675324675)
--» processing VACJ elec 47 / 48 freq 0_theta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.9777777777777779 (pvalue : 0.000999000999000999)
VACJ ['xpow', 'label', 'channel', 'xyz'] xpow shape:  [(48, 12), (48, 12)]
--» processing VACJ elec 0 / 48 freq 1_alpha
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.5833333333333334 (pvalue : 0.4225774225774226)
--» processing VACJ elec 1 / 48 freq 1_alpha
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.37222222222222223 (pvalue : 0.7672327672327672)
--» processing VACJ elec 2 / 48 freq 1_alpha
Size of the concatenated data:  (24, 1)
Size of label for cl

Classification score 0.8111111111111111 (pvalue : 0.022977022977022976)
--» processing VACJ elec 43 / 48 freq 1_alpha
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.6055555555555555 (pvalue : 0.34965034965034963)
--» processing VACJ elec 44 / 48 freq 1_alpha
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.6555555555555556 (pvalue : 0.26073926073926074)
--» processing VACJ elec 45 / 48 freq 1_alpha
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.6833333333333333 (pvalue : 0.13486513486513488)
--» processing VACJ elec 46 / 48 freq 1_alpha
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.7666666666666667 (pvalue : 0.060939060939060936)
--» processing VACJ elec 47 / 48 freq 1_alpha
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.7555555555555555 (pvalue : 0.080919

Classification score 0.7055555555555555 (pvalue : 0.15984015984015984)
--» processing VACJ elec 39 / 48 freq 2_beta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.16666666666666669 (pvalue : 0.9590409590409591)
--» processing VACJ elec 40 / 48 freq 2_beta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.5666666666666667 (pvalue : 0.33166833166833165)
--» processing VACJ elec 41 / 48 freq 2_beta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.8 (pvalue : 0.04295704295704296)
--» processing VACJ elec 42 / 48 freq 2_beta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.711111111111111 (pvalue : 0.12487512487512488)
--» processing VACJ elec 43 / 48 freq 2_beta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.7444444444444445 (pvalue : 0.06693306693306693)
--» proces

Classification score 0.6222222222222222 (pvalue : 0.3246753246753247)
--» processing VACJ elec 36 / 48 freq 3_gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.7277777777777777 (pvalue : 0.12387612387612387)
--» processing VACJ elec 37 / 48 freq 3_gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.3555555555555555 (pvalue : 0.7322677322677322)
--» processing VACJ elec 38 / 48 freq 3_gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.611111111111111 (pvalue : 0.34465534465534464)
--» processing VACJ elec 39 / 48 freq 3_gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.1 (pvalue : 0.996003996003996)
--» processing VACJ elec 40 / 48 freq 3_gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.38333333333333336 (pvalue : 0.6973026973026973)
--» proce

...

Classification score 0.26666666666666666 (pvalue : 0.9310689310689311)
--» processing PIRJ elec 25 / 30 freq 2_beta
Size of the concatenated data:  (28, 1)
Size of label for classif:  28
Classification score 0.7333333333333333 (pvalue : 0.08691308691308691)
--» processing PIRJ elec 26 / 30 freq 2_beta
Size of the concatenated data:  (28, 1)
Size of label for classif:  28
Classification score 0.42777777777777776 (pvalue : 0.6063936063936064)
--» processing PIRJ elec 27 / 30 freq 2_beta
Size of the concatenated data:  (28, 1)
Size of label for classif:  28
Classification score 0.8111111111111111 (pvalue : 0.02197802197802198)
--» processing PIRJ elec 28 / 30 freq 2_beta
Size of the concatenated data:  (28, 1)
Size of label for classif:  28
Classification score 0.6722222222222223 (pvalue : 0.18581418581418582)
--» processing PIRJ elec 29 / 30 freq 2_beta
Size of the concatenated data:  (28, 1)
Size of label for classif:  28
Classification score 0.65 (pvalue : 0.3116883116883117)
PIRJ ['xp

Classification score 0.7555555555555555 (pvalue : 0.1048951048951049)
--» processing FERJ elec 9 / 45 freq 0_theta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.85 (pvalue : 0.023976023976023976)
--» processing FERJ elec 10 / 45 freq 0_theta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.8388888888888889 (pvalue : 0.008991008991008992)
--» processing FERJ elec 11 / 45 freq 0_theta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.05 (pvalue : 1.0)
--» processing FERJ elec 12 / 45 freq 0_theta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.5055555555555555 (pvalue : 0.45254745254745254)
--» processing FERJ elec 13 / 45 freq 0_theta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.5555555555555556 (pvalue : 0.34965034965034963)
--» processing FERJ elec 14 / 45 

Classification score 0.7944444444444445 (pvalue : 0.055944055944055944)
--» processing FERJ elec 8 / 45 freq 1_alpha
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.6611111111111111 (pvalue : 0.25674325674325676)
--» processing FERJ elec 9 / 45 freq 1_alpha
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.6833333333333333 (pvalue : 0.24975024975024976)
--» processing FERJ elec 10 / 45 freq 1_alpha
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.11666666666666667 (pvalue : 0.988011988011988)
--» processing FERJ elec 11 / 45 freq 1_alpha
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.7611111111111111 (pvalue : 0.06293706293706294)
--» processing FERJ elec 12 / 45 freq 1_alpha
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.35 (pvalue : 0.7072927072927073)
--» pr

Classification score 0.55 (pvalue : 0.35564435564435565)
--» processing FERJ elec 7 / 45 freq 2_beta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.6777777777777778 (pvalue : 0.15684315684315683)
--» processing FERJ elec 8 / 45 freq 2_beta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.7666666666666667 (pvalue : 0.06493506493506493)
--» processing FERJ elec 9 / 45 freq 2_beta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.23888888888888887 (pvalue : 0.9120879120879121)
--» processing FERJ elec 10 / 45 freq 2_beta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.5833333333333334 (pvalue : 0.43956043956043955)
--» processing FERJ elec 11 / 45 freq 2_beta
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.6611111111111111 (pvalue : 0.25374625374625376)
--» process

Classification score 0.5611111111111111 (pvalue : 0.3176823176823177)
--» processing FERJ elec 7 / 45 freq 3_gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.7944444444444445 (pvalue : 0.029970029970029972)
--» processing FERJ elec 8 / 45 freq 3_gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.2 (pvalue : 0.965034965034965)
--» processing FERJ elec 9 / 45 freq 3_gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.9333333333333333 (pvalue : 0.000999000999000999)
--» processing FERJ elec 10 / 45 freq 3_gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.6 (pvalue : 0.3996003996003996)
--» processing FERJ elec 11 / 45 freq 3_gamma
Size of the concatenated data:  (24, 1)
Size of label for classif:  24
Classification score 0.6722222222222223 (pvalue : 0.21678321678321677)
--» processing FERJ elec